This code was written using some libraries that require Pytorch versions pre-2.0, and some other tools. Installing these first, and was written to run in Google Colab.

This is _research code_ and not meant for production use. No warranty or support will be provided.

In [ ]:
!pip install torch==1.13.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 32.5 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.13.1 which

In [ ]:
!pip install -e git+https://github.com/garydoranjr/misvm.git#egg=misvm

Obtaining misvm from git+https://github.com/garydoranjr/misvm.git#egg=misvm
  Updating ./src/misvm clone
  Running command git fetch -q --tags
  Running command git reset --hard -q b2118fe04d98c00436bdf8a0e4bbfb6082c5751c
  Preparing metadata (setup.py) ... done
  Attempting uninstall: misvm
    Found existing installation: misvm 1.0
    Uninstalling misvm-1.0:
      Successfully uninstalled misvm-1.0
  Running setup.py develop for misvm


In [ ]:
import misvm

In [ ]:
!pip install git+https://github.com/ml-jku/hopfield-layers #This needs older Python :(

  Cloning https://github.com/ml-jku/hopfield-layers to /tmp/pip-req-build-ln3tpo9t
  Running command git clone --filter=blob:none --quiet https://github.com/ml-jku/hopfield-layers /tmp/pip-req-build-ln3tpo9t
  Resolved https://github.com/ml-jku/hopfield-layers to commit f56f929c95b77a070ae675ea4f56b6d54d36e730
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cpu.html

Looking in links: https://data.pyg.org/whl/torch-1.13.0+cpu.html


In [ ]:
from scipy.io import loadmat
import numpy as np

In [ ]:
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import string
import pathlib
import gzip
from numba import jit

from tqdm.autonotebook import tqdm

import random
import heapq
import math

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import fbeta_score


In [ ]:
from torch import Tensor
from torch_geometric.nn import GCNConv, global_mean_pool

In [ ]:
from hflayers import Hopfield
from hflayers import HopfieldLayer
from hflayers import HopfieldPooling

In [ ]:
# Algorithm 1 Single-Concept Standard-MIL
def falseNegativeMIL(train_size, test_size, min_size=None, d=16):
    labels_train = []
    bags_train = []

    #Training set
    for _ in range(train_size//2): #Generate the negative instances
        bag = []
        #True MIL should not be able to learn to use this poison item!
        poison = np.random.normal(-10, 0.1, size=d)
        bag.append(poison)
        #Background items contain no info
        if min_size is None:
          for i in range(np.random.randint(1, 10)):
            bag.append(np.random.normal(0.0, 1.0, size=d))
        else:
          while(len(bag) < min_size):
            bag.append(np.random.normal(0.0, 1.0, size=d))

        labels_train.append(0)
        bags_train.append(np.vstack(bag))

    for _ in range(train_size//2): #Generate the positive instances
        bag = []


        #poisitive indiactors
        for i in range(np.random.randint(1, 4)):
            if np.random.randint(0,1) == 0:#Subtle positive indiactor
                bag.append(np.random.normal(0.0, 3.0, size=d))
            else: #more obv positive indiactor
                bag.append(np.random.normal(1.0, 1.0, size=d))


        #Background items contain no info
        if min_size is None:
          for i in range(np.random.randint(1, 5)):
            bag.append(np.random.normal(0.0, 1.0, size=d))
        else:
          while len(bag) < min_size:
            bag.append(np.random.normal(0.0, 1.0, size=d))

        labels_train.append(1)
        bags_train.append(np.vstack(bag))

    #Test set, poison gets put in the _positive_ bags now, which should cause no issues if using a real MIL model
    labels_test = []
    bags_test = []
    for _ in range(test_size//2): #Generate the negative instances
        bag = []
        #Background items contain no info
        if min_size is None:
          for i in range(np.random.randint(1, 10)):
            bag.append(np.random.normal(0.0, 1.0, size=d))
        else:
          while(len(bag) < min_size):
            bag.append(np.random.normal(0.0, 1.0, size=d))


        labels_test.append(0)
        bags_test.append(np.vstack(bag))

    for _ in range(test_size//2): #Generate the positive instances
        bag = []

        #True MIL should not be able to learn to use this poison item!
        poison = np.random.normal(-10, 0.1, size=d)
        bag.append(poison)

        #poisitive indiactors
        for i in range(np.random.randint(1, 4)):
            if np.random.randint(0,1) == 0:#Subtle positive indiactor
                bag.append(np.random.normal(0.0, 3.0, size=d))
            else: #more obv positive indiactor
                bag.append(np.random.normal(1.0, 1.0, size=d))

        #Background items contain no info
        if min_size is None:
          for i in range(np.random.randint(1, 5)):
            bag.append(np.random.normal(0.0, 1.0, size=d))
        else:
          while len(bag) < min_size:
            bag.append(np.random.normal(0.0, 1.0, size=d))

        labels_test.append(1)
        bags_test.append(np.vstack(bag))

    return bags_train, np.array(labels_train, dtype=np.float32), bags_test, np.array(labels_test, dtype=np.float32)


In [ ]:
# Algorithm 2 Multi-Concept Standard-MIL
# The "v2" argument tests a change in the ratio of options put in the bag, see OpenReview discussion.
def falseConceptPairMIL(train_size, test_size, min_size=None, d=16, v2=False):
    labels_train = []
    bags_train = []

    #Training set
    for _ in range(train_size//2): #Generate the negative instances
        bag = []

        #True MIL should not be able to learn to use this poison item!
        poison = np.random.normal(-10, 0.1, size=d)
        bag.append(poison)

        #include only one of A or B, but not both
        choice = np.random.randint(0, 2)
        if v2:
          choice = np.random.randint(0, 3)
        if choice == 0:
            #Item A
            bag.append(np.random.normal(2, 0.1, size=d))
        #Item B
        elif choice == 1:
            bag.append(np.random.normal(3, 0.1, size=d))

        #Background items contain no info
        if min_size is None:
          for i in range(np.random.randint(1, 5)):
            bag.append(np.random.normal(0.0, 1.0, size=d))
        else:
          while len(bag) < min_size:
            bag.append(np.random.normal(0.0, 1.0, size=d))

        labels_train.append(0)
        bags_train.append(np.vstack(bag))

    for _ in range(train_size//2): #Generate the positive instances
        bag = []

        #True MIL should not be able to learn to use this poison item!
        if np.random.randint(0, 10) == 0:
          poison = np.random.normal(-10, 0.1, size=d)
          bag.append(poison)

        #Item A
        for i in range(np.random.randint(1, 5)):
            bag.append(np.random.normal(2, 0.1, size=d))
        #Item B
        for i in range(np.random.randint(1, 5)):
            bag.append(np.random.normal(3, 0.1, size=d))

        #Background items contain no info
        if min_size is None:
          for i in range(np.random.randint(1, 5)):
            bag.append(np.random.normal(0.0, 1.0, size=d))
        else:
          while len(bag) < min_size:
            bag.append(np.random.normal(0.0, 1.0, size=d))

        labels_train.append(1)
        bags_train.append(np.vstack(bag))

    #Test set, negative instnaces have way too many copies of just _one_ item, but not both. Should still be negative, unless learned count instead of groups
    labels_test = []
    bags_test = []
    for _ in range(test_size//2): #Generate the negative instances
        bag = []

        #include only one of A or B, but not both
        choice = np.random.randint(0, 2)
        if choice == 0:
            #Item A
            bag.append(np.random.normal(2, 0.1, size=d))
        #Item B
        elif choice == 1:
            bag.append(np.random.normal(3, 0.1, size=d))

        #Background items contain no info
        if min_size is None:
          for i in range(np.random.randint(1, 5)):
            bag.append(np.random.normal(0.0, 1.0, size=d))
        else:
          while len(bag) < min_size:
            bag.append(np.random.normal(0.0, 1.0, size=d))

        labels_test.append(0)
        bags_test.append(np.vstack(bag))

    for _ in range(test_size//2): #Generate the positive instances
        bag = []

        #True MIL should not be able to learn to use this poison item!
        poison = np.random.normal(-10, 0.1, size=d)
        bag.append(poison)

        #Item A
        for i in range(np.random.randint(1, 5)):
            bag.append(np.random.normal(2, 0.1, size=d))
        #Item B
        for i in range(np.random.randint(1, 5)):
            bag.append(np.random.normal(3, 0.1, size=d))

        #Background items contain no info
        if min_size is None:
          for i in range(np.random.randint(1, 5)):
            bag.append(np.random.normal(0.0, 1.0, size=d))
        else:
          while len(bag) < min_size:
            bag.append(np.random.normal(0.0, 1.0, size=d))

        labels_test.append(1)
        bags_test.append(np.vstack(bag))

    return bags_train, np.array(labels_train, dtype=np.float32), bags_test, np.array(labels_test, dtype=np.float32)

In [ ]:
# This is algorithm 3 from the paper.
def falseFrequencyMIL(train_size, test_size, min_size=None, d=16):
    labels_train = []
    bags_train = []

    #Training set
    for _ in range(train_size//2): #Generate the negative instances
        bag = []

        #include only one of A or B, but not both
        choice = np.random.randint(0, 2)
        if choice == 0:
          #Item A
          for i in range(np.random.randint(1,2)):
            bag.append(np.random.normal(-2, 0.1, size=d))
        #Item B
        elif choice == 1:
          for i in range(np.random.randint(1,2)):
            bag.append(np.random.normal(2, 0.1, size=d))

        #Background items contain no info
        if min_size is None:
          for i in range(np.random.randint(1, 10)):
            bag.append(np.random.normal(0.0, 1.0, size=d))
        else:
          while(len(bag) < min_size):
            bag.append(np.random.normal(0.0, 1.0, size=d))
        labels_train.append(0)
        bags_train.append(np.vstack(bag))

    for _ in range(train_size//2): #Generate the positive instances
        bag = []

        #Item A
        for i in range(np.random.randint(1, 2)):
            bag.append(np.random.normal(-2, 0.1, size=d))
        #Item B
        for i in range(np.random.randint(1, 2)):
            bag.append(np.random.normal(2, 0.1, size=d))

        #Background items contain no info
        if min_size is None:
          for i in range(np.random.randint(1, 10)):
            bag.append(np.random.normal(0.0, 1.0, size=d))
        else:
          while(len(bag) < min_size):
            bag.append(np.random.normal(0.0, 1.0, size=d))

        labels_train.append(1)
        bags_train.append(np.vstack(bag))

    #Test set, negative instnaces have way too many copies of just _one_ item, but not both. Should still be negative, unless learned count instead of groups
    labels_test = []
    bags_test = []
    for _ in range(test_size//2): #Generate the negative instances
        bag = []

        if np.random.randint(1, 10) % 2 == 0:
          #Item A
          for i in range(np.random.randint(35, 40)):
            bag.append(np.random.normal(-2, 0.1, size=d))
        else:
          #Item B
          for i in range(np.random.randint(35, 40)):
            bag.append(np.random.normal(2, 0.1, size=d))

        #Background items contain no info
        if min_size is None:
          for i in range(np.random.randint(1, 10)):
            bag.append(np.random.normal(0.0, 1.0, size=d))
        else:
          while(len(bag) < min_size):
            bag.append(np.random.normal(0.0, 1.0, size=d))

        labels_test.append(0)
        bags_test.append(np.vstack(bag))

    for _ in range(test_size//2): #Generate the positive instances
        bag = []

        #poisitive indiactors
        #Item A
        for i in range(np.random.randint(1, 2)):
            bag.append(np.random.normal(-2, 0.1, size=d))
        #Item B
        for i in range(np.random.randint(1, 2)):
            bag.append(np.random.normal(2, 0.1, size=d))

        #Background items contain no info
        if min_size is None:
          for i in range(np.random.randint(1, 10)):
            bag.append(np.random.normal(0.0, 1.0, size=d))
        else:
          while(len(bag) < min_size):
            bag.append(np.random.normal(0.0, 1.0, size=d))

        labels_test.append(1)
        bags_test.append(np.vstack(bag))

    return bags_train, np.array(labels_train, dtype=np.float32), bags_test, np.array(labels_test, dtype=np.float32)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
class MILScaler():
    def __init__(self, scaler):
        self.scaler = scaler

    def fit(self, X):
        return self.scaler.fit(np.vstack(X))

    def transform(self, X):
        return [self.scaler.transform(x) for x in X]

In [ ]:
# X_train, y_train, X_test, y_test = falseNegativeMIL(1000,1000)#, min_size=min_size)
# X_train, y_train, X_test, y_test =  falseFrequencyMIL(1000,1000)
X_train, y_train, X_test, y_test = falseConceptPairMIL(200,1000, v2=True)

names = [
    "mi-SVM", "MI-SVM", "SIL", "NSK", "STK",
         "MICA", "MissSVM"
         ]
gamma = 0.1000
C = 1.0
models = [
  misvm.miSVM(kernel='linear', C=C, gamma=gamma, scale_C=False, max_iters=250, verbose =False),
  misvm.MISVM(kernel='linear', C=C, gamma=gamma, scale_C=False, max_iters=250, verbose =False),
  misvm.SIL(kernel='linear', C=C,  gamma=gamma, scale_C=False, verbose =False, sv_cutoff=1e-15),
  misvm.NSK(kernel='linear', C=C,  gamma=gamma, scale_C=False, verbose =False, sv_cutoff=1e-15),
  misvm.STK(kernel='linear', C=C, gamma=gamma, scale_C=False, verbose =False, sv_cutoff=1e-15),
  # misvm.MICA(kernel='linear', C=1.0, gamma=gamma, scale_C=False, max_iters=250, verbose =False),
  # misvm.MissSVM(kernel='linear', C=1.0, gamma=gamma, scale_C=False, max_iters=250, verbose =False)

  # misvm.miSVM(kernel='rbf', C=C, gamma=gamma, scale_C=False, max_iters=250, verbose =False),
  # misvm.MISVM(kernel='rbf', C=C, gamma=gamma, scale_C=False, max_iters=250, verbose =False),
  # misvm.SIL(kernel='rbf', C=C,  gamma=gamma, scale_C=False, verbose =False),
  # misvm.NSK(kernel='rbf', C=C,  gamma=gamma, scale_C=False, verbose =False),
  # misvm.STK(kernel='rbf', C=C, gamma=gamma, scale_C=False, verbose =False),
  # misvm.MICA(kernel='rbf', C=1.0, gamma=gamma, scale_C=False, max_iters=250, verbose =False),
  # misvm.MissSVM(kernel='rbf', C=1.0, gamma=gamma, scale_C=False, max_iters=250, verbose =False)
]

for name, classifier in zip(names, models):
  classifier.fit(X_train, y_train*2-1)
  preds_train = classifier.predict(X_train)
  preds = classifier.predict(X_test)

  result = {
    "Train: Balanced Accuracy":balanced_accuracy_score(y_train, np.round((np.sign(preds_train)+1)/2)),
    "Train: Accuracy":accuracy_score(y_train, np.round((np.sign(preds_train)+1)/2)),
    "Train: F1":f1_score(y_train, np.round((np.sign(preds_train)+1)/2)),
    "Train: AUC":roc_auc_score(y_train, preds_train),
    #####
    "Balanced Accuracy":balanced_accuracy_score(y_test, np.round((np.sign(preds)+1)/2)),
    "Accuracy":accuracy_score(y_test, np.round((np.sign(preds)+1)/2)),
    "F1":f1_score(y_test, np.round((np.sign(preds)+1)/2)),
    "AUC":roc_auc_score(y_test, preds),
    }
  print(name, result)

In [ ]:
class MILDataset(torch.utils.data.Dataset):

    def __init__(self, X, y, scaler=None):

        self.X = X
        self.y = y
        if scaler is None:
            self.scaler = StandardScaler()
#             self.scaler = MinMaxScaler()
            self.scaler.fit(np.vstack(X))
        else:
            self.scaler = scaler

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):

        x = self.scaler.transform(np.vstack(self.X[idx]))
        y = int(self.y[idx])

        return [torch.tensor(q, dtype=torch.float32) for q in x], torch.tensor(y, dtype=torch.int64)


In [ ]:
max_trials = 100

In [ ]:
NUM_GPUS = 1
MAX_BAG_SIZE = 5000//4  #trim all bags to this many strings
MAX_BATCH_STRINGS = 5000 #max desired number of effective strings in a batch (includes padding strings! )
MAX_BATCH_SIZE = 32
# MAX_BATCH_SIZE = 1

In [ ]:
class VariableBatchSampler(torch.utils.data.Sampler):
    """Wraps another sampler to yield a mini-batch of indices.

    Args:
        sampler (Sampler): Base sampler.
        batch_size (int): Size of mini-batch.
        drop_last (bool): If ``True``, the sampler will drop the last batch if
            its size would be less than ``batch_size``

    Example:
        >>> list(BatchSampler(SequentialSampler(range(10)), batch_size=3, drop_last=False))
        [[0, 1, 2], [3, 4, 5], [6, 7, 8], [9]]
        >>> list(BatchSampler(SequentialSampler(range(10)), batch_size=3, drop_last=True))
        [[0, 1, 2], [3, 4, 5], [6, 7, 8]]
    """

    def __init__(self, bag_sizes, NUM_GPUS=1, max_batch_size=32):
        """
        bag_sizes: for eah index i in a source dataset, bag_sizes[i] should
            indicate the number of strings in that bag.
        """

        finished = set()

        iter_order = np.argsort(bag_sizes)[::-1] #reverse order, big items first please


        #grab single items that are already at the max size, will be a batch size of 1
        #finished.update(iter_order[bag_sizes[iter_order] >= MAX_BATCH_STRINGS].tolist())

        #Now lets grab the leftover items
        iter_order = iter_order[bag_sizes[iter_order] < MAX_BATCH_STRINGS]

        final_bag_batches = [ ]

        #a heap as tuples (priority, value)
        #priority is the bagBatchSize, smallest bag batches will come out first
        to_improve = [ ( -min(bag_sizes[i],MAX_BAG_SIZE) , i) for i in iter_order]
        heapq.heapify(to_improve)

        #"Rounded" size, where we round based on numb GPUs and clip to MAX_BAG_SIZE and MAX_BATCH_STRINGS/NUM_GPUS
        #This ensures that the batches we create can be distributed cleanly accros NUM_GPUS without any bag's
        #contents stretching across a GPU boundary
        def rSize(x):
            return min(math.ceil(x/float(NUM_GPUS))*NUM_GPUS, MAX_BAG_SIZE, MAX_BATCH_STRINGS//NUM_GPUS)

        while True and len(to_improve) > 0:
            _, toAdd = to_improve[0]
            cur_list = [toAdd]
            heapq.heappop(to_improve)

            #While we can add one more smaller bag, to it
            while len(cur_list) < max_batch_size and len(to_improve) > 0 and rSize(max(bag_sizes[cur_list]))*rSize(len(cur_list)+1) <= MAX_BATCH_STRINGS:
                _, toAdd = to_improve[0]
                cur_list = cur_list + [toAdd]
                heapq.heappop(to_improve)
            finished.update(cur_list)
            final_bag_batches.append( cur_list )
        self.final_bag_batches = final_bag_batches

    def __iter__(self):
        order = list(range(len(self)))
        random.shuffle(order)
        for i in order:
#             print(self.final_bag_batches[i])
            yield self.final_bag_batches[i]

    def __len__(self):
        return len(self.final_bag_batches)


In [ ]:
PAD_VAL = 0

def randomSlice(arr, max_size):
    if len(arr) <= max_size:
        return np.pad(arr,(0,max_size-len(arr)),'constant', constant_values=(PAD_VAL, PAD_VAL))[0:max_size]
        return arr
    start = np.random.randint(0,len(arr)-max_size)
    return (arr[start:start+max_size])[0:max_size]

def randomSubset(bag, max_size):
    if len(bag) <= max_size:
        return bag

    return np.random.choice(bag, min(len(bag),max_size), replace=False)

ignore_index = -100 #Default for softmax cross entropy

def collate_fn(batch):
    #X = [randomSubset(bag,MAX_BAG_SIZE) for bag, _ in batch]
    X = [bag for bag, _ in batch]
    Y = [y for _, y in batch]

    #longest string within the batch, clipped to max we wil allow
    batch_max = max([max([len(s) for s in bag]) for bag, _ in batch])

    #We want the first index of the array to be a generic empy vector for use in the model's inference process
    #we are doing that here to maximize GPU util so this can be done in advance and not be a GPU op that is
    #almost pure overhead costs
    empty = np.array([PAD_VAL]*batch_max, dtype=np.float32)

    #Figure out how many items are in the biggest bag, and padd it out based on num GPUs
    bag_max_size = max([len(x) for x in X])
    bag_max_size = math.ceil(bag_max_size/float(NUM_GPUS))*NUM_GPUS
    #pad the bags so that we can evenly split the bag into parts
    for x in X:
        while len(x) < bag_max_size:
            x.append( empty )
    #Pad out the batch itself!
    while len(X) != math.ceil(len(X)/float(NUM_GPUS))*NUM_GPUS:
        X.append([empty]*bag_max_size)
        Y.append(ignore_index)
    B = len(X)

    X = np.vstack(
    [
        np.vstack([randomSlice(s, batch_max) for s in bag])
        for bag in X]
    )

    batch_starts = np.array([i*bag_max_size for i in range(B)], dtype=np.int64)
    return X, batch_starts, Y #, weird

In [ ]:
# collate_fn([mildata[2], mildata[72]])[0].shape

In [ ]:
class AttentionAvg(nn.Module):

    def __init__(self, attnScore):
        super(AttentionAvg, self).__init__()
        self.score = attnScore

    def forward(self, states, context=None, mask=None):
        """
        states: (B, T, D) shape
        context: (B, D) shape
        output: (B, D), a weighted av

        """

        B = states.size(0)
        T = states.size(1)
        D = states.size(2)

        #If we were not given a context, but we do have a mask, create context ourselves
        if context is None and mask is not None:
            context = states.sum(dim=1)/(mask.sum(dim=1).unsqueeze(1)+1e-5) #(B, D)

        scores = self.score(states, context) #(B, T, 1)

        if mask is not None:
            scores[~mask] = float(-10000)
        weights = F.softmax(scores, dim=1) #(B, T, 1) still, but sum(T) = 1

        context = (states*weights).sum(dim=1) #(B, T, D) * (B, T, 1) -> (B, D, 1)

#         [z for z in X.shape[0:-2]] + [X.size(-1)]
        return context.view(B, D) #Flatten this out to (B, D)


class AdditiveAttentionScore(nn.Module):

    def __init__(self, D):
        super(AdditiveAttentionScore, self).__init__()
        self.v = nn.Linear(D, 1)
        self.w = nn.Linear(2*D, D)

    def forward(self, states, context):
        """
        states:  (B, *, T, D) shape
        context: (B, *, D) shape
        output:  (B, *, T, 1), giving a score to each of the T items based on the context D

        """
        T = states.size(-2)
        #Repeating the values T times
        context = torch.stack([context for _ in range(T)], dim=len(context.shape)-1) #(B, *, D) -> (B, *, T, D)
        state_context_combined = torch.cat((states, context), dim=-1) #(B, *, T, D) + (B, *, T, D)  -> (B, *, T, 2*D)
        scores = self.v(torch.tanh(self.w(state_context_combined)))
        return scores

class GeneralScore(nn.Module):

    def __init__(self, D):
        super(GeneralScore, self).__init__()
        self.w = nn.Bilinear(D, D, 1)

    def forward(self, states, context):
        """
        states: (B, T, D) shape
        context: (B, D) shape
        output: (B, T, 1), giving a score to each of the T items based on the context D

        """
        T = states.size(1)
        D = states.size(2)
        #Repeating the values T times
        context = torch.stack([context for _ in range(T)], dim=1) #(B, D) -> (B, T, D)
        scores = self.w(states, context) #(B, T, D) -> (B, T, 1)
        return scores

class DotScore(nn.Module):

    def __init__(self, D):
        super(DotScore, self).__init__()

    def forward(self, states, context):
        """
        states: (B, T, D) shape
        context: (B, D) shape
        output: (B, T, 1), giving a score to each of the T items based on the context D

        """
        T = states.size(1)
        D = states.size(2)

        scores = torch.bm

In [ ]:
class View(nn.Module):
    def __init__(self, *shape):
        super(View, self).__init__()
        self.shape = shape

    def forward(self, input):
        return input.view(*self.shape)

In [ ]:
def getMaskByFill(x, time_dimension=1, fill=0):
    """
    x: the original input with three or more dimensions, (B, ..., T, ...)
        which may have unsued items in the tensor. B is the batch size,
        and T is the time dimension.
    time_dimension: the axis in the tensor `x` that denotes the time dimension
    fill: the constant used to denote that an item in the tensor is not in use,
        and should be masked out (`False` in the mask).

    return: A boolean tensor of shape (B, T), where `True` indicates the value
        at that time is good to use, and `False` that it is not.
    """
    to_sum_over = list(range(1,len(x.shape))) #skip the first dimension 0 because that is the batch dimension

    if time_dimension in to_sum_over:
        to_sum_over.remove(time_dimension)

    with torch.no_grad():
        #Special case is when shape is (B, D), then it is an embedding layer. We just return the values that are good
        if len(to_sum_over) == 0:
            return (x != fill)
        #(x!=fill) determines locations that might be unused, beause they are
        #missing the fill value we are looking for to indicate lack of use.
        #We then count the number of non-fill values over everything in that
        #time slot (reducing changes the shape to (B, T)). If any one entry
        #is non equal to this value, the item represent must be in use -
        #so return a value of true.
        mask = torch.sum((x != fill), dim=to_sum_over) > 0
    return mask

In [ ]:
class BatchModel(nn.Module):

    def __init__(self, d_in, D, out_size=2):
        super(BatchModel, self).__init__()
        leak=0.2
        self.emb = nn.Sequential(
            nn.Linear(d_in, D),
            nn.LeakyReLU(leak),
            nn.Dropout(),
            nn.Linear(D, D),
            nn.LeakyReLU(leak),
            nn.Dropout(),
        )
        self.attn = AttentionAvg(AdditiveAttentionScore(D))
        self.pred = nn.Sequential(
            nn.Linear(D, D),
            nn.LeakyReLU(leak),
            nn.Dropout(),
            nn.Linear(D, D),
            nn.LeakyReLU(leak),
            nn.Dropout(),
            nn.Linear(D, out_size),
        )

    def forward(self, input_strings, batch_starts):
        """
        input_strings: a tensor of shape (S+1, T), where S is the total number of strings in each bag,
            and T is the length of the longest string. The first row should contain only padding values.
        batch_starts: a tensor of shape (B+1), where B is the total number of bags/batch size.
            For 0 <= i <= B, batch_starts[i] should contain the index in `input_strings` that the i'th
            bag starts at. Everything between [batch_starts[i], batch_starts[i+1]) should belong to the
            i'th bag. The last index at batch_starts[B] should be the value S+1.
        """

        device = input_strings.device
        z = self.emb(input_strings) #(S, D') -> (S, D)

        bag_sizes = batch_starts[1:]-batch_starts[0:-1] # (B)
        max_bag_size = torch.max(bag_sizes) #We are going to pad out smaller bags to the largest bag size

        ###
        #z is currently shaped (S, D) which is (B*L, D), B = batch_starts.size
        #So can view as (B, L, D)
        new_batch = z.view(batch_starts.size(0), -1, z.size(1))

        mask = getMaskByFill(new_batch)

        z = self.attn(new_batch, mask=mask) #(B, D)

        return self.pred(z)

In [ ]:
class MIL_Pool(nn.Module):

    def __init__(self, d_in, D, out_size=1):
        super(MIL_Pool, self).__init__()
        leak=0.2
        self.emb = nn.Sequential(
            nn.Linear(d_in, D),
            nn.LeakyReLU(leak),
            nn.Dropout(),
            nn.Linear(D, D),
            nn.LeakyReLU(leak),
            nn.Dropout(),
            nn.Linear(D, D),
            nn.LeakyReLU(leak),
            nn.Dropout(),
            # nn.Linear(D, out_size),
            # nn.LeakyReLU(leak),
            # nn.Dropout(),
        )
        self.attn = AttentionAvg(AdditiveAttentionScore(D))
        self.pred = nn.Sequential(
#             nn.Linear(D, D),
#             nn.LeakyReLU(leak),
#             nn.Dropout(),
#             nn.Linear(D, D),
#             nn.LeakyReLU(leak),
#             nn.Dropout(),
            nn.Linear(D, out_size),
        )

    def forward(self, input_strings, batch_starts):
        """
        input_strings: a tensor of shape (S+1, T), where S is the total number of strings in each bag,
            and T is the length of the longest string. The first row should contain only padding values.
        batch_starts: a tensor of shape (B+1), where B is the total number of bags/batch size.
            For 0 <= i <= B, batch_starts[i] should contain the index in `input_strings` that the i'th
            bag starts at. Everything between [batch_starts[i], batch_starts[i+1]) should belong to the
            i'th bag. The last index at batch_starts[B] should be the value S+1.
        """

        device = input_strings.device
        z = self.emb(input_strings) #(S, D') -> (S, D)
        # print(z.shape)
        bag_sizes = batch_starts[1:]-batch_starts[0:-1] # (B)
        max_bag_size = torch.max(bag_sizes) #We are going to pad out smaller bags to the largest bag size
        # print(bag_sizes, max_bag_size)
        ###
        #z is currently shaped (S, D) which is (B*L, D), B = batch_starts.size
        #So can view as (B, L, D)
        new_batch = z.view(batch_starts.size(0), -1, z.size(-1))
        # print(new_batch.shape)
        mask = getMaskByFill(new_batch)

        new_batch = torch.where(mask.view(batch_starts.size(0), -1, 1), new_batch, torch.full_like(new_batch, -1000.0))

        # print(new_batch.permute(0, 2, 1).shape)
        new_batch = self.attn(new_batch, mask=mask) #(B, D)
        # new_batch = F.adaptive_max_pool1d(z.permute(0, 2, 1), 1).view(-1, 1)
        return new_batch.sum(-1).view(-1, 1)

#         print(new_batch.shape)

        # return z #self.pred(z)

In [ ]:
class miNet(nn.Module):

    def __init__(self, d_in, D):
        super(miNet, self).__init__()
        leak=0.2
        self.emb = nn.Sequential(
            nn.Linear(d_in, D),
            nn.LeakyReLU(leak),
            nn.Dropout(),
            nn.Linear(D, D),
            nn.LeakyReLU(leak),
            nn.Dropout(),
            nn.Linear(D, D),
            nn.LeakyReLU(leak),
            nn.Dropout(),
            nn.Linear(D, 1)
        )

        # self.pred = nn.Sequential(
        #     nn.Linear(D, 1)
        # )

    def forward(self, input_strings, batch_starts):
        """
        input_strings: a tensor of shape (S+1, T), where S is the total number of strings in each bag,
            and T is the length of the longest string. The first row should contain only padding values.
        batch_starts: a tensor of shape (B+1), where B is the total number of bags/batch size.
            For 0 <= i <= B, batch_starts[i] should contain the index in `input_strings` that the i'th
            bag starts at. Everything between [batch_starts[i], batch_starts[i+1]) should belong to the
            i'th bag. The last index at batch_starts[B] should be the value S+1.
        """

        device = input_strings.device
        z = self.emb(input_strings) #(S, D') -> (S, 1)

        bag_sizes = batch_starts[1:]-batch_starts[0:-1] # (B)
        max_bag_size = torch.max(bag_sizes) #We are going to pad out smaller bags to the largest bag size

        ###
        #z is currently shaped (S, D) which is (B*L, D), B = batch_starts.size
        #So can view as (B, L, D)
        new_batch = z.view(batch_starts.size(0), -1, z.size(1))

        mask = getMaskByFill(new_batch)

        final_context = F.adaptive_max_pool1d(new_batch.permute(0, 2, 1), 1).squeeze() #(B, T, D) -> (B, D)

        return final_context.view(-1, 1)

In [ ]:
class MiNet(nn.Module):

    def __init__(self, d_in, D):
        super(MiNet, self).__init__()
        leak=0.2
        self.emb = nn.Sequential(
            nn.Linear(d_in, D),
            nn.LeakyReLU(leak),
            nn.Dropout(),
            nn.Linear(D, D),
            nn.LeakyReLU(leak),
            nn.Dropout(),
            nn.Linear(D, D),
            nn.LeakyReLU(leak),
            nn.Dropout(),
        )

        self.pred = nn.Sequential(
            nn.Linear(D, 1)
        )

    def forward(self, input_strings, batch_starts):
        """
        input_strings: a tensor of shape (S+1, T), where S is the total number of strings in each bag,
            and T is the length of the longest string. The first row should contain only padding values.
        batch_starts: a tensor of shape (B+1), where B is the total number of bags/batch size.
            For 0 <= i <= B, batch_starts[i] should contain the index in `input_strings` that the i'th
            bag starts at. Everything between [batch_starts[i], batch_starts[i+1]) should belong to the
            i'th bag. The last index at batch_starts[B] should be the value S+1.
        """

        device = input_strings.device
        z = self.emb(input_strings) #(S, D') -> (S, D)

        bag_sizes = batch_starts[1:]-batch_starts[0:-1] # (B)
        max_bag_size = torch.max(bag_sizes) #We are going to pad out smaller bags to the largest bag size

        ###
        #z is currently shaped (S, D) which is (B*L, D), B = batch_starts.size
        #So can view as (B, L, D)
        new_batch = z.view(batch_starts.size(0), -1, z.size(1))

        mask = getMaskByFill(new_batch)

        final_context = F.adaptive_max_pool1d(new_batch.permute(0, 2, 1), 1).squeeze() #(B, T, D) -> (B, D)

        return self.pred(final_context)

In [ ]:
class TranMil(nn.Module):

    def __init__(self, d_in, D):
        super(TranMil, self).__init__()
        leak=0.2
        self.emb = nn.Sequential(
            nn.Linear(d_in, D),
        )
        self.layers = nn.Sequential(
            nn.TransformerEncoderLayer(d_model=D, dim_feedforward=D, nhead=8, batch_first=True),
            nn.TransformerEncoderLayer(d_model=D, dim_feedforward=D, nhead=8, batch_first=True),
            nn.TransformerEncoderLayer(d_model=D, dim_feedforward=D, nhead=8, batch_first=True),
        )

        self.pred = nn.Sequential(
            nn.Linear(D, 1)
        )

    def forward(self, input_strings, batch_starts):
        """
        input_strings: a tensor of shape (S+1, T), where S is the total number of strings in each bag,
            and T is the length of the longest string. The first row should contain only padding values.
        batch_starts: a tensor of shape (B+1), where B is the total number of bags/batch size.
            For 0 <= i <= B, batch_starts[i] should contain the index in `input_strings` that the i'th
            bag starts at. Everything between [batch_starts[i], batch_starts[i+1]) should belong to the
            i'th bag. The last index at batch_starts[B] should be the value S+1.
        """

        device = input_strings.device
        z = self.emb(input_strings) #(S, D') -> (S, D)

        bag_sizes = batch_starts[1:]-batch_starts[0:-1] # (B)
        max_bag_size = torch.max(bag_sizes) #We are going to pad out smaller bags to the largest bag size

        ###
        #z is currently shaped (S, D) which is (B*L, D), B = batch_starts.size
        #So can view as (B, L, D)
        new_batch = z.view(batch_starts.size(0), -1, z.size(1))
        new_batch = self.layers(new_batch)

        mask = getMaskByFill(new_batch)

        final_context = F.adaptive_max_pool1d(new_batch.permute(0, 2, 1), 1).squeeze() #(B, T, D) -> (B, D)

        return self.pred(final_context)

In [ ]:
class LinearNN(nn.Module):
    __constants__ = ['in_features', 'out_features']

    def __init__(self, in_features, out_features, bias=True):
        super(LinearNN, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.Tensor(out_features, in_features))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        self.weight.data = torch.log(torch.abs(self.weight.data))
        if self.bias is not None:
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / np.math.sqrt(fan_in)
            nn.init.uniform_(self.bias, -bound, bound)

    def forward(self, input):
        return F.linear(input, torch.exp(self.weight), self.bias)

    def extra_repr(self):
        return 'in_features={}, out_features={}, bias={}'.format(
            self.in_features, self.out_features, self.bias is not None
        )

In [ ]:
class MILAttention(nn.Module):

    def __init__(self, D):
        super(MILAttention, self).__init__()
        self.score = MonoAdditiveAttentionScore(D, D)
        self.pool = CountMILPool(D)

    def forward(self, states, context=None, mask=None):
        """
        states: (B, T, D) shape
        context: (B, D) shape
        output: (B, D), a weighted av

        """

        B = states.size(0)
        T = states.size(1)
        D = states.size(2)

        #If we were not given a context, but we do have a mask, create context ourselves
        if context is None and mask is not None:
            #context = states.sum(dim=1)/(mask.sum(dim=1).unsqueeze(1)+1e-5) #(B, D)
            #context = F.adaptive_max_pool1d(states.permute(0, 2, 1), 1).squeeze() #(B, D)
            context = self.pool(states)

#         print("T:", states.shape, context.shape)
        scores = self.score(states, context) #(B, T, 1)
#         print(scores.shape)
        if mask is not None:
            scores[~mask] = float(-10000)
        #weights = F.softmax(scores, dim=1) #(B, T, 1) still, but sum(T) = 1
        weights = torch.sigmoid(scores)

#         print(states.shape, weights.shape)
        final_context = (states*weights) #(B, T, D) * (B, T, 1) -> (B, T, D)
        #final_context = F.adaptive_max_pool1d(final_context.permute(0, 2, 1), 1).squeeze() #(B, T, D) -> (B, D)
        final_context = self.pool(final_context)

#         [z for z in X.shape[0:-2]] + [X.size(-1)]
        return final_context.view(B, -1) #Flatten this out to (B, D)
#         return context.view(B, D) #Flatten this out to (B, D)

In [ ]:
class NMModel(nn.Module):

    def __init__(self, d_in, D):
        super(NMModel, self).__init__()
        leak=0.2
        self.emb = nn.Sequential(
            nn.Linear(d_in, D),
            nn.LeakyReLU(leak),
            nn.Dropout(),
            nn.Linear(D, D),
            nn.LeakyReLU(leak),
            nn.Dropout(),
        )
        self.attn = MILAttention(D)
        self.pred = MonotonicG(D, 10, D//10)

    def forward(self, input_strings, batch_starts):
        """
        input_strings: a tensor of shape (S+1, T), where S is the total number of strings in each bag,
            and T is the length of the longest string. The first row should contain only padding values.
        batch_starts: a tensor of shape (B+1), where B is the total number of bags/batch size.
            For 0 <= i <= B, batch_starts[i] should contain the index in `input_strings` that the i'th
            bag starts at. Everything between [batch_starts[i], batch_starts[i+1]) should belong to the
            i'th bag. The last index at batch_starts[B] should be the value S+1.
        """

        device = input_strings.device
        z = self.emb(input_strings) #(S, D') -> (S, D)

        bag_sizes = batch_starts[1:]-batch_starts[0:-1] # (B)
        max_bag_size = torch.max(bag_sizes) #We are going to pad out smaller bags to the largest bag size

        ###
        #z is currently shaped (S, D) which is (B*L, D), B = batch_starts.size
        #So can view as (B, L, D)
        new_batch = z.view(batch_starts.size(0), -1, z.size(1))

        mask = getMaskByFill(new_batch)

        z = torch.sigmoid(self.attn(new_batch, mask=mask)) #(B, D)

        return self.pred(z)

In [ ]:
data = torch.randn(7, 10)
pair_dists = torch.cdist(data, data)
order = torch.topk(pair_dists, 3, largest=False).indices
edges = torch.hstack((torch.stack((order[:,0],order[:,1])), torch.stack((order[:,0],order[:,2]))))

In [ ]:
class MilGCN(torch.nn.Module):
  def __init__(self, d_in, D):
    super(MilGCN, self).__init__()
    self.conv1 = GCNConv(d_in, D)
    self.conv2 = GCNConv(D, D)
    self.conv3 = GCNConv(D, D)
    self.lin = nn.Linear(D, 1)

  def forward(self, input_strings, batch_starts):
    batch = 0
    device = input_strings.device
    x = input_strings #(L, D) - because we are hard coding to a bath size of 1 for this model

    # print(z.shape)

    # bag_sizes = batch_starts[1:]-batch_starts[0:-1] # (B)
    # max_bag_size = torch.max(bag_sizes) #We are going to pad out smaller bags to the largest bag size

    ###
    #z is currently shaped (S, D) which is (B*L, D), B = batch_starts.size
    #So can view as (B, L, D)
    # new_batch = z.view(batch_starts.size(0), -1, z.size(1))
    x = x
    # print(x.shape)
    pair_dists = torch.cdist(x, x)
    order = torch.topk(pair_dists, 3, largest=False).indices #create graph based on 2-NN
    edge_index = torch.hstack((torch.stack((order[:,0],order[:,1])), torch.stack((order[:,0],order[:,2]))))
    #X comes in with shape ()
    # 1. Obtain node embeddings
    x = self.conv1(x, edge_index)
    x = x.relu()
    x = self.conv2(x, edge_index)
    x = x.relu()
    x = self.conv3(x, edge_index)

    # 2. Readout layer
    x = global_mean_pool(x, torch.zeros(x.size(0), dtype=torch.long))  # [batch_size, D]

    # 3. Apply a final classifier
    x = F.dropout(x, p=0.1, training=self.training)
    x = self.lin(x)

    return x

# Run a Network

In [ ]:
epochs = 20

In [ ]:
embed_dim = 128
hidden_dim = 128
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
oneItemOnly = False
scores_dataset = {}


min_size = 45

# X_train, y_train, X_test, y_test = falseNegativeMIL(10000,1000)#, min_size=min_size)
# # X_train, y_train, X_test, y_test =  falseFrequencyMIL(10000,1000, min_size=min_size)
X_train, y_train, X_test, y_test = falseConceptPairMIL(10000,1000, min_size=min_size, v2=True)

X_train_v, X_v, y_train_v, y_v = train_test_split(X_train, y_train,
                                                        test_size=0.2, random_state=42, stratify=y_train)

# scalar_train = MILScaler(MinMaxScaler())
# scalar_train_v = MILScaler(MinMaxScaler())
scalar_train = MILScaler(StandardScaler())
scalar_train_v = MILScaler(StandardScaler())

scalar_train.fit(X_train)
scalar_train_v.fit(X_train_v)

#     print(len(X_train_v[0]))

train_dataset = MILDataset(X_train, y_train)
test_dataset = MILDataset(X_test, y_test, scaler=train_dataset.scaler)

BATCH_SIZE = 1

trainSampler = VariableBatchSampler(np.array([len(x) for x in X_train_v]), max_batch_size=BATCH_SIZE, NUM_GPUS=NUM_GPUS)

train_loader = DataLoader(train_dataset, num_workers=2, collate_fn=collate_fn,
                     batch_sampler=VariableBatchSampler(np.array([len(x) for x in X_train_v]),
                        max_batch_size=BATCH_SIZE, NUM_GPUS=NUM_GPUS))
test_loader = DataLoader(test_dataset, num_workers=2, collate_fn=collate_fn,
                     batch_sampler=VariableBatchSampler(np.array([len(x) for x in X_test]),
                        max_batch_size=BATCH_SIZE, NUM_GPUS=NUM_GPUS))


d = train_dataset[0][0][0].shape[0]

#First set of Neural Network options, uncomment whichever one you want to run

#model = BatchModel(d, hidden_dim)
# model = MiNet(d, hidden_dim)
# model = miNet(d, hidden_dim)
# model = MIL_Pool(d, hidden_dim)
# model = TranMil(d, hidden_dim)
model = MilGCN(d, hidden_dim)

#Multiple versions f the Hopfield network that do not work. You need to uncomment “oneItemOnly” too, as the code doesn’t work with batches at a time.
# oneItemOnly = True
# hopfield_pooling = HopfieldPooling(input_size=d, hidden_size=128, num_heads = 8)
# output_projection = nn.Linear(in_features=hopfield_pooling.output_size, out_features=1)
# model = nn.Sequential(hopfield_pooling, output_projection, nn.Flatten(start_dim=0))
# hopfield = Hopfield(input_size=d, hidden_size=128, num_heads=8, update_steps_max=3, scaling=0.25, dropout=0.5)
# output_projection = nn.Linear(in_features=hopfield.output_size * min_size, out_features=1)
# model = nn.Sequential(hopfield, nn.Flatten(), output_projection, nn.Flatten(start_dim=0)).to(device=device)
# hopfield = Hopfield(input_size=d, hidden_size=128, num_heads=8, update_steps_max=3, scaling=0.25, dropout=0.5)
# hopfield_pooling = HopfieldPooling(input_size=hopfield.output_size, hidden_size=128, num_heads = 8)
# output_projection = nn.Linear(in_features=hopfield_pooling.output_size, out_features=1)
# model = nn.Sequential(hopfield, hopfield_pooling, output_projection, nn.Flatten(start_dim=0)).to(device=device)

opt = torch.optim.AdamW(model.parameters())
criteria = nn.BCEWithLogitsLoss()

def fixGrad(grad):
    grad[torch.isnan(grad)] = 0
    return torch.clamp(grad, -5, 5)

for p in model.parameters():
    p.register_hook(fixGrad)


results_per_epoch = {}
model = model.to(device)
for epoch in tqdm(range(epochs), desc="Epoch"):
    opt.zero_grad()
    pred_labels = []
    true_labels = []
    scores = []

    model = model.train()
    for batch, batch_starts, labels in tqdm(train_loader, leave=False):
        batch = torch.tensor(batch, dtype=torch.float32, device=device)
        batch_starts = torch.tensor(batch_starts, dtype=torch.int64, device=device)
#         labels = torch.tensor(labels, dtype=torch.int64, device=device)
        labels = torch.tensor(labels, dtype=torch.float32, device=device).view(-1,1)

        if BATCH_SIZE == 1 and oneItemOnly:
          output = model(batch.view(1, batch.size(0), -1))
          labels = labels.view(1)
        else:
          output = model(batch, batch_starts)

        with torch.no_grad():
            pred_label = (1*(torch.sigmoid(output) > 0.5)).cpu().numpy().tolist()

            pred_labels.extend(pred_label)
            true_labels.extend(labels.cpu().numpy().tolist())
            scores.extend(output.cpu().numpy().tolist())

        loss = criteria(output, labels)
        loss.backward()
        opt.step()
        opt.zero_grad()
    #Eval Time!
    model = model.eval()

    pred_labels_eval = []
    true_labels_eval = []
    scores_eval = []
    for batch, batch_starts, labels in tqdm(test_loader, leave=False):
        batch = torch.tensor(batch, dtype=torch.float32, device=device)
        batch_starts = torch.tensor(batch_starts, dtype=torch.int64, device=device)
        labels = torch.tensor(labels, dtype=torch.int64, device=device)

        with torch.no_grad():
            if BATCH_SIZE == 1 and oneItemOnly:
              output = model(batch.view(1, batch.size(0), -1))
              labels = labels.view(1)
            else:
              output = model(batch, batch_starts)

            pred_label = (1*(torch.sigmoid(output) > 0.5)).cpu().numpy().tolist()
            scores_eval.extend(output.cpu().numpy().tolist())

            pred_labels_eval.extend(pred_label)
            true_labels_eval.extend(labels.cpu().numpy().tolist())


    scores = np.array(scores)
    true_labels = np.array(true_labels).flatten()
    pred_labels = np.array(pred_labels)
    scores_eval = np.array(scores_eval)
    true_labels_eval = np.array(true_labels_eval)
    pred_labels_eval = np.array(pred_labels_eval)

    scores = scores[true_labels >= 0]
    pred_labels = pred_labels[true_labels >= 0]
    true_labels = true_labels[true_labels >= 0]
    scores_eval = scores_eval[true_labels_eval >= 0]
    pred_labels_eval = pred_labels_eval[true_labels_eval >= 0]
    true_labels_eval = true_labels_eval[true_labels_eval >= 0]

    results_per_epoch[epoch] = {
        "Train: Balanced Accuracy":balanced_accuracy_score(true_labels, pred_labels),
        "Train: Accuracy":accuracy_score(true_labels, pred_labels),
        "Train: F1":f1_score(true_labels, pred_labels),
        "Train: AUC":roc_auc_score(true_labels, scores),
        #####
        "Balanced Accuracy":balanced_accuracy_score(true_labels_eval, pred_labels_eval),
        "Accuracy":accuracy_score(true_labels_eval, pred_labels_eval),
        "F1":f1_score(true_labels_eval, pred_labels_eval),
        "AUC":roc_auc_score(true_labels_eval, scores_eval),
    }
#Done training
print(results_per_epoch[epochs-1])
